In [1]:
import numpy as np
import matplotlib.pyplot as plt
import utilis as u
import importlib
import model as m 

Using TensorFlow backend.


In [2]:
importlib.reload(u)
filename = '/data_batch_1'
X_train, y_train,Y_train = u.load_data(filename, reshape=False, clipping=True)
meanX = np.mean(X_train,axis=1)
stdX = np.std(X_train,axis=1)
X_train = (X_train-meanX.reshape((len(meanX),1)))/stdX.reshape((len(stdX),1))

filename = '/data_batch_2'
X_val, y_val,Y_val = u.load_data(filename, reshape=False, clipping=True)
X_val = (X_val-meanX.reshape((len(meanX),1)))/stdX.reshape((len(stdX),1))

filename = '/test_batch'
X_test, y_test,Y_test = u.load_data(filename, reshape=False, clipping=True)
X_test = (X_test-meanX.reshape((len(meanX),1)))/stdX.reshape((len(stdX),1))

data = {'X_train':X_train, 'Y_train':Y_train, 'y_train':y_train,'X_val':X_val, 'Y_val':Y_val, 'y_val':y_val, 'X_test':X_test, 'Y_test':Y_test, 'y_test':y_test}

In [4]:
importlib.reload(m)
mlp = m.MLP(lambda_=1e-3, dimensions=[100,60,30,10])
P = mlp.forward_batchnorm(X_train[:100,:1])
mlp.compute_gradients_batchnorm(X_train[:100,:1], Y_train[:,:1],P)
mlp.compare_gradients(X_train[:100,:1], Y_train[:,:1])

In [2]:
importlib.reload(m)
X_train_whole, y_train_whole, Y_train_whole = u.load_data('/data_batch_1', clipping=True)
for i in range(2,6):
    X, y, Y = u.load_data('/data_batch_'+str(i), clipping=True)
    X_train_whole = np.concatenate((X, X_train_whole), axis=1)
    y_train_whole = np.concatenate((y, y_train_whole))
    Y_train_whole = np.concatenate((Y, Y_train_whole), axis=1)

X_val_small, y_val_small, Y_val_small = X_train_whole[:,-5000:], y_train_whole[-5000:], Y_train_whole[:,-5000:]
X_train_whole, y_train_whole, Y_train_whole = X_train_whole[:,:-5000], y_train_whole[:-5000], Y_train_whole[:,:-5000]

filename = '/test_batch'
X_test_45, y_test,Y_test = u.load_data(filename, reshape=False, clipping=True)

## normalize with mean and std of train set 
mean = np.mean(X_train_whole, axis=1)
std = np.std(X_train_whole, axis=1)

X_train_whole -= np.outer(mean, np.ones(X_train_whole.shape[1]))
X_train_whole /= np.outer(std, np.ones(X_train_whole.shape[1]))

X_val_small -= np.outer(mean, np.ones(X_val_small.shape[1]))
X_val_small /= np.outer(std, np.ones(X_val_small.shape[1]))

X_test_45 -= np.outer(mean, np.ones(X_test_45.shape[1]))
X_test_45 /= np.outer(std, np.ones(X_test_45.shape[1]))


data_45 = {'X_train':X_train_whole, 'Y_train':Y_train_whole, 'y_train':y_train_whole,'X_val':X_val_small, 'Y_val':Y_val_small, 'y_val':y_val_small, 'X_test':X_test_45, 'Y_test':Y_test,'y_test':y_test}

# Train 3 layer Neural Networks

In [3]:
importlib.reload(m)
ns = 5*45000/100
GD_params = {"n_batch":100, "eta_min":1e-5, 'eta_max':1e-1, 'ns':ns, 'n_cycles':2, 'freq':10}
dims = [3072,50,50,10]
for seed in range(3):
    mlp = m.MLP(lambda_=5e-3, dimensions=dims,seed=seed,layer_init='He')
    model = mlp.cyclicLearning(data_45, GD_params,True, '3_layer_with_batchnorm', False, True)
    mlp = m.MLP(lambda_=5e-3, dimensions=dims,seed=seed,layer_init='He')
    model = mlp.cyclicLearning(data_45, GD_params,False, '3_layer_without_batchnorm', False, True)

100%|██████████| 20/20 [06:40<00:00, 20.02s/it]


# Train 9 Layer Neural Networks

In [4]:
importlib.reload(m)
ns = 5*45000/100
GD_params = {"n_batch":100, "eta_min":1e-5, 'eta_max':1e-1, 'ns':ns, 'n_cycles':2, 'freq':10}
dims = [3072,50,30,20,20,10,10,10,10]
for seed in range(3):
    mlp = m.MLP(lambda_=5e-3, dimensions=dims,seed=seed,layer_init='He')
    model = mlp.cyclicLearning(data_45, GD_params,True, '9_layer_with_batchnorm', False, True)
    mlp = m.MLP(lambda_=5e-3, dimensions=dims,seed=seed,layer_init='He')
    model = mlp.cyclicLearning(data_45, GD_params,False, '9_layer_without_batchnorm', False, True)

100%|██████████| 20/20 [07:19<00:00, 21.99s/it]


# Test for initialisation sensitivity

In [5]:
importlib.reload(m)
ns = 5*45000/100
GD_params = {"n_batch":100, "eta_min":1e-5, 'eta_max':1e-1, 'ns':ns, 'n_cycles':2, 'freq':10}
dims = [3072,50,50,10]
for init in [1e-1,1e-3,1e-4]:
    mlp = m.MLP(lambda_=5e-3, dimensions=dims,layer_init=init)
    model = mlp.cyclicLearning(data_45, GD_params,True, f'init_{init}_with_batchnorm', False, True)
    mlp = m.MLP(lambda_=5e-3, dimensions=dims,layer_init=init)
    model = mlp.cyclicLearning(data_45, GD_params,False, f'init_{init}_without_batchnorm', False, True)

100%|██████████| 20/20 [05:53<00:00, 17.68s/it]


# Search for best $\lambda$

In [6]:
importlib.reload(m)
ns = 5*45000/100
GD_params = {"epochs":2, "n_batch":100, "eta_min":1e-5,'eta_max':1e-1, 'ns':ns, 'n_cycles':2, 'freq':10}
search = m.LambdaSearch(-5, -1,10)
model = search.lambda_search(data=data_45,GDparams=GD_params)

100%|██████████| 20/20 [07:33<00:00, 22.66s/it]


In [19]:
m = list(model.keys())
print(f'{m[0]} & {m[1]} & {m[2]} & {m[3]} & {m[4]} & {m[5]} & {m[6]} & {m[7]} & {m[8]} & {m[9]} \\')
print(f'{np.round(model[m[0]].lambda_,5)} & {np.round(model[m[1]].lambda_,5)} & {np.round(model[m[2]].lambda_,5)} & {np.round(model[m[3]].lambda_,5)} & {np.round(model[m[4]].lambda_,5)} & {np.round(model[m[5]].lambda_,5)} & {np.round(model[m[6]].lambda_,5)} & {np.round(model[m[7]].lambda_,5)} & {np.round(model[m[8]].lambda_,5)} & {np.round(model[m[9]].lambda_,5)} \\')

0.5048 & 0.5094 & 0.509 & 0.511 & 0.5206 & 0.52 & 0.5234 & 0.527 & 0.512 & 0.4952 \
1e-05 & 3e-05 & 8e-05 & 0.00022 & 0.0006 & 0.00167 & 0.00464 & 0.01292 & 0.03594 & 0.1 \


In [23]:
import model as m 
importlib.reload(m)
ns = 5*45000/100
GD_params = {"epochs":2, "n_batch":100, "eta_min":1e-5,'eta_max':1e-1, 'ns':ns, 'n_cycles':2, 'freq':10}
search = m.LambdaSearch(-3, -1,10)
model = search.lambda_search(data=data_45,GDparams=GD_params)

100%|██████████| 20/20 [06:46<00:00, 20.31s/it]


In [26]:
m = list(model.keys())
print(f'{m[0]} & {m[1]} & {m[2]} & {m[3]} & {m[4]} & {m[5]} & {m[6]} & {m[7]} & {m[8]} \\')
print(f'{np.round(model[m[0]].lambda_,5)} & {np.round(model[m[1]].lambda_,5)} & {np.round(model[m[2]].lambda_,5)} & {np.round(model[m[3]].lambda_,5)} & {np.round(model[m[4]].lambda_,5)} & {np.round(model[m[5]].lambda_,5)} & {np.round(model[m[6]].lambda_,5)} & {np.round(model[m[7]].lambda_,5)} & {np.round(model[m[8]].lambda_,5)} \\')

0.5202 & 0.52 & 0.529 & 0.5234 & 0.5336 & 0.527 & 0.512 & 0.5002 & 0.4952 \
0.001 & 0.00167 & 0.02154 & 0.00464 & 0.00774 & 0.01292 & 0.03594 & 0.05995 & 0.1 \


# Train with the best $\lambda$

In [29]:
importlib.reload(m)
X_train_whole, y_train_whole, Y_train_whole = u.load_data('/data_batch_1', clipping=True)
for i in range(2,6):
    X, y, Y = u.load_data('/data_batch_'+str(i), clipping=True)
    X_train_whole = np.concatenate((X, X_train_whole), axis=1)
    y_train_whole = np.concatenate((y, y_train_whole))
    Y_train_whole = np.concatenate((Y, Y_train_whole), axis=1)

X_val_small, y_val_small, Y_val_small = X_train_whole[:,-1000:], y_train_whole[-1000:], Y_train_whole[:,-1000:]
X_train_whole, y_train_whole, Y_train_whole = X_train_whole[:,:-1000], y_train_whole[:-1000], Y_train_whole[:,:-1000]

filename = '/test_batch'
X_test_45, y_test,Y_test = u.load_data(filename, reshape=False, clipping=True)

## normalize with mean and std of train set 
mean = np.mean(X_train_whole, axis=1)
std = np.std(X_train_whole, axis=1)

X_train_whole -= np.outer(mean, np.ones(X_train_whole.shape[1]))
X_train_whole /= np.outer(std, np.ones(X_train_whole.shape[1]))

X_val_small -= np.outer(mean, np.ones(X_val_small.shape[1]))
X_val_small /= np.outer(std, np.ones(X_val_small.shape[1]))

X_test_45 -= np.outer(mean, np.ones(X_test_45.shape[1]))
X_test_45 /= np.outer(std, np.ones(X_test_45.shape[1]))


data_49 = {'X_train':X_train_whole, 'Y_train':Y_train_whole, 'y_train':y_train_whole,'X_val':X_val_small, 'Y_val':Y_val_small, 'y_val':y_val_small, 'X_test':X_test_45, 'Y_test':Y_test,'y_test':y_test}

In [32]:
import model as m 
importlib.reload(m)
ns = 5*49000/100
GD_params = {"n_batch":100, "eta_min":1e-5, 'eta_max':1e-1, 'ns':ns, 'n_cycles':4, 'freq':10}
dims = [3072,50,50,10]

mlp = m.MLP(lambda_=0.00774, dimensions=dims,layer_init='He')
model = mlp.cyclicLearning(data_49, GD_params,True, 'best_lambda', False, True)



  0%|          | 0/40 [00:00<?, ?it/s]

  2%|▎         | 1/40 [00:23<15:11, 23.37s/it]

  5%|▌         | 2/40 [00:46<14:44, 23.26s/it]

  8%|▊         | 3/40 [01:09<14:19, 23.22s/it]

 10%|█         | 4/40 [01:32<13:51, 23.11s/it]

 12%|█▎        | 5/40 [01:53<13:08, 22.54s/it]

 15%|█▌        | 6/40 [02:20<13:31, 23.88s/it]

 18%|█▊        | 7/40 [02:42<12:49, 23.31s/it]

 20%|██        | 8/40 [03:04<12:11, 22.86s/it]

 22%|██▎       | 9/40 [03:25<11:37, 22.48s/it]

 25%|██▌       | 10/40 [03:53<11:55, 23.85s/it]

 28%|██▊       | 11/40 [04:16<11:24, 23.59s/it]

 30%|███       | 12/40 [04:41<11:14, 24.08s/it]

 32%|███▎      | 13/40 [05:03<10:39, 23.67s/it]

 35%|███▌      | 14/40 [05:29<10:28, 24.18s/it]

 38%|███▊      | 15/40 [05:51<09:47, 23.52s/it]

 40%|████      | 16/40 [06:14<09:18, 23.28s/it]

 42%|████▎     | 17/40 [06:36<08:53, 23.19s/it]

 45%|████▌     | 18/40 [07:02<08:46, 23.93s/it]

 48%|████▊     | 19/40 [07:24<08:06, 23.16s/it]

 50%|█████     | 20/40 [07:49<07:54,